# 📓 MEE2024 Notebook Version

## Modern Eddington Experiment - Jupyter Notebook Conversion

**Version:** 1.0  
**Last Updated:** 2024  
**Original GUI Version:** MEE2024 v0.6.0  
**Notebook Format:** Jupyter Notebook (.ipynb)

---

## 📖 Overview

This notebook converts the MEE2024 GUI into an interactive Jupyter notebook format. It performs the same analysis as the original GUI, but in a step-by-step format that's easier to understand, modify, and learn from.

**What This Notebook Does:**

This notebook measures **Einstein's gravitational deflection coefficient** by analyzing star positions during a solar eclipse. The analysis follows a three-step pipeline:

1. **Step 1: Find Centroids** - Stacks images, finds star positions, and does plate solving
2. **Step 2: Compute Distortion** - Matches stars to Gaia catalog and fits optical distortion model  
3. **Step 3: Eclipse Analysis** - Calculates gravitational deflection and finds Einstein's deflection coefficient

**Expected Result:** The deflection coefficient L should be approximately **1.751 arcseconds** (Einstein's prediction).

---

## 🚀 How to Use This Notebook

### Prerequisites

- **MEE2024 package** installed (in virtual environment)
- **Python 3.9+**
- All dependencies from `requirements.txt`
- **Internet connection** (needed for Gaia catalog queries)

### Quick Start

1. **Run the Setup cell** (Cell 2) - This imports libraries and initializes the star catalog database
2. **Configure your files** (Cell 4) - Set paths to your image files
3. **Run Step 1** - Process images and find star centroids
4. **Run Step 2** - Fit distortion model using Gaia catalog
5. **Run Step 3** - Calculate deflection coefficient

### Detailed Workflow

#### Step 1: Find Centroids
1. Set your image file paths in the configuration cell
2. Optionally add dark and flat frame paths
3. Run all cells in Step 1 section
4. **Note the output ZIP file name** (e.g., `centroid_data20240101120000.zip`)

#### Step 2: Compute Distortion
1. The notebook will automatically use the Step 1 output file
2. Adjust distortion fitting options if needed
3. Run all cells in Step 2 section
4. **Note the output ZIP file name** (e.g., `distortion_data20240101120000.zip`)

#### Step 3: Eclipse Analysis
1. The notebook will automatically use the Step 2 output file
2. Adjust eclipse analysis options if needed
3. Run all cells in Step 3 section
4. **View your final results** - The deflection coefficient L will be displayed

### Tips

- **File paths:** Use absolute paths (full path from root) for best results
- **Output directory:** Set an output directory to keep results organized
- **First run:** The database initialization may take a few minutes (one-time setup)
- **Internet required:** Step 2 needs internet to query the Gaia catalog

---

## 📚 Notebook Structure

This notebook is organized into clear sections:

```
📁 Setup & Configuration
   ├── Import libraries
   ├── Initialize database
   └── Configure file paths and options

📁 Step 1: Find Centroids
   ├── Configuration
   ├── File validation
   ├── Image stacking & centroid detection
   └── Results display

📁 Step 2: Compute Distortion
   ├── Configuration
   ├── Plate solving & Gaia matching
   ├── Distortion fitting
   └── Results display

📁 Step 3: Eclipse Analysis
   ├── Configuration
   ├── Deflection calculation
   ├── Coefficient fitting
   └── Final results

📁 Additional Resources
   ├── Troubleshooting guide
   └── Testing checklist
```

---

## ⚠️ Important Notes

- **This is a direct conversion of the GUI** - Future adaptations for coronagraph analysis will be added later
- **Run cells in order** - Each step depends on the previous step's output
- **Save your work** - Output files are saved to disk, but notebook state should be saved regularly
- **Check file paths** - Make sure all file paths are correct before running each step

---

## 📝 Credits & References

**Original MEE2024 Software:**  
- Author: Andrew Smith  
- Version: 0.6.0 (May 2024)  
- Repository: https://github.com/andrew551/MEE2024

**Notebook Conversion:**  
- Converted from GUI to Jupyter notebook format for educational use
- Enhanced with detailed comments and explanations

**Scientific Background:**  
- Modern Eddington Experiment (MEE2024)
- Measures gravitational light deflection during solar eclipses
- Tests Einstein's general theory of relativity

---

---

## 🔧 Setup: Import Libraries and Initialize

First, we import all the necessary libraries and initialize the database.

**⏱️ Note:** The database initialization may take a few minutes the first time you run this notebook. Subsequent runs will be much faster.

**✅ What this cell does:**
- Imports all required Python packages
- Initializes the star catalog database (for plate solving)
- Sets up visualization tools

**💡 Tip:** Run this cell first, and only once per notebook session.

---

In [ ]:
# ============================================================================
# IMPORTS: Load all necessary libraries
# ============================================================================

# Standard library imports
import os
import sys
from pathlib import Path
import json
import zipfile

# Scientific computing
import numpy as np
import pandas as pd

# Astronomy libraries
from astropy.io import fits
from astropy.coordinates import EarthLocation, SkyCoord
from astropy.time import Time

# MEE2024 package imports
# These are the core functions that do the actual work
from mee2024 import stacker_implementation  # For Step 1: image stacking
from mee2024 import distortion_fitter       # For Step 2: distortion fitting
from mee2024 import eclipse_analysis         # For Step 3: deflection calculation
from mee2024 import MEE2024util              # Utility functions
from mee2024 import database_cache          # Star catalog database

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline  # Display plots in notebook

# ============================================================================
# INITIALIZE DATABASE: Prepare star catalog for plate solving
# ============================================================================
# This loads the star catalog database (one-time setup, may take a minute)
print("Initializing star catalog database...")
database_cache.prepare_triangles()
print("✅ Database ready!")

---

## ⚙️ Configuration: Set Your File Paths and Options

**⚠️ IMPORTANT:** Change the file paths below to point to your actual image files!

**📋 Quick Setup:**
1. Set `light_files` - Your eclipse images (required)
2. Optionally set `dark_files` - For noise reduction
3. Optionally set `flat_files` - For vignetting correction
4. Set `output_directory` - Where results will be saved (optional, defaults to input directory)
5. Adjust `options` dictionary - Most users can leave defaults

---

In [ ]:
# ============================================================================
# CONFIGURATION: Set your file paths and processing options
# ============================================================================
# 
# INSTRUCTIONS:
# 1. Replace the empty lists below with paths to your image files
# 2. Use absolute paths (full path from root) or paths relative to this notebook
# 3. You can use Python list syntax: ['/path/to/file1.fits', '/path/to/file2.fits']
# ============================================================================

# File paths - CHANGE THESE to point to your images
light_files = [
    # Example format (uncomment and modify):
    # '/Users/yourname/images/image1.fits',
    # '/Users/yourname/images/image2.fits',
    # '/Users/yourname/images/image3.fits',
]

dark_files = [
    # Optional: Add dark frame paths here (for noise reduction)
    # Example: '/Users/yourname/images/dark1.fits',
]

flat_files = [
    # Optional: Add flat frame paths here (for vignetting correction)
    # Example: '/Users/yourname/images/flat1.fits',
]

# Output directory - where results will be saved
# Leave empty ('') to save in same folder as input images
output_directory = ''

# ============================================================================
# PROCESSING OPTIONS: Adjust these settings as needed
# ============================================================================
# Most users can leave these at default values
# ============================================================================

options = {
    # Display and output options
    'flag_display': True,              # Show graphics/plots during processing
    'save_dark_flat': False,            # Save stacked dark/flat images
    'float_fits': False,                # Save 32-bit float FITS files (larger but more precise)
    
    # Image stacking options
    'sensitive_mode_stack': True,       # Use sensitive mode for finding dimmer stars (slower but more accurate)
    'd': 100,                           # Number of brightest stars to display in plots
    
    # Sun/Moon masking options (important for eclipse/coronagraph images)
    'delete_saturated_blob': True,      # Remove saturated Sun/Moon region
    'blob_saturation_level': 95,        # Saturation threshold (% of max pixel value)
    'blob_radius_extra': 100,           # Extra pixels to exclude around saturated blob
    'centroid_gap_blob': 30,            # Ignore centroids within this distance of blob
    
    # Centroid detection options
    'centroid_gaussian_subtract': False, # Use "sensitive mode" for centroid detection
    'centroid_gaussian_thresh': 5.0,    # Threshold for detecting centroids (sensitive mode)
    'min_area': 4,                      # Minimum area for found centroids (pixels)
    'sigma_subtract': 3.0,              # Background subtraction parameter
    'background_subtraction_mode': 'annular',  # Background method: 'Gaussian' or 'annular'
    'remove_edgy_centroids': True,      # Remove centroids near image edges
    
    # Advanced options (usually leave at defaults)
    'img_edge_distance': 5,             # Pixels away from edge to exclude
    'pxl_tol': 10,                      # Pixel tolerance for stacking
    'cutoff': 100,                      # Penalty saturation distance
    'sanity_check_centroids': True,     # Perform sanity checks on centroids
    'experimental_background_subtract': False,
    
    # Catalog options
    'catalogue': 'gaia',                # Star catalog to use: 'gaia' or 'tycho'
    'max_star_mag_dist': 12,            # Maximum star magnitude for distortion fitting
    'safety_limit_mag': 13,             # Safety limit for star magnitude
    
    # Date and location (needed for Step 2 if using corrections)
    'observation_date': '2023-12-01',   # Observation date (YYYY-MM-DD format)
    'observation_time': '',             # Observation time (HH:MM:SS UTC, optional)
    'observation_lat': '',            # Observation latitude (degrees, optional)
    'observation_long': '',            # Observation longitude (degrees, optional)
    'guess_date': False,                # Auto-guess observation date
    
    # Distortion fitting options (used in Step 2)
    'distortionOrder': 'cubic',         # Polynomial order: 'linear', 'cubic', 'quintic', 'septic'
    'distortion_fit_tol': 1.0,          # Fit tolerance (arcseconds)
    'rough_match_threshhold': 36,       # Rough match threshold (arcseconds)
    'distortion_reference_files': '',   # Reference distortion files (optional)
    'distortion_fixed_coefficients': 'None',  # Fix certain coefficients
    
    # Correction options (used in Step 2)
    'enable_corrections': False,         # Enable aberration and parallax corrections
    'enable_corrections_ref': False,     # Enable refraction correction
    'enable_gravitational_def': False,  # Enable gravitational deflection correction
    'observation_temp': 10,             # Temperature (°C) for refraction
    'observation_pressure': 1010,        # Pressure (millibars) for refraction
    'observation_humidity': 0,          # Humidity (0.0 to 1.0) for refraction
    'observation_height': 0,            # Height above sea level (meters)
    'observation_wavelength': 0.65,     # Wavelength (micrometers) for refraction
    
    # Eclipse analysis options (used in Step 3)
    'eclipse_limiting_mag': 11,         # Maximum star magnitude for eclipse analysis
    'eclipse_method': 'Method 1 & 2',   # Analysis method: 'Method 1', 'Method 2', or 'Method 1 & 2'
    'limit_radial_sun_radii': False,    # Limit stars by distance from Sun
    'limit_radial_sun_radii_value': 9,  # Maximum distance in solar radii
    'remove_double_stars_eclipse': False, # Remove double stars from analysis
    'object_centre_moon': False,        # Center on Moon instead of Sun
    'gravity_sweep': False,             # Simultaneous deflection and platescale fit
    'crop_circle': False,                # Crop to circular region
    'crop_circle_thresh': 1.0,          # Crop circle threshold
    'remove_double_tab2': False,         # Remove double stars in Step 2
    
    # Internal options (usually don't change)
    'workDir': '',                       # Working directory (auto-set)
    'workDir2': '',                      # Working directory 2 (auto-set)
    '-DARK-': '',                        # Dark files string (auto-set)
    '-FLAT-': '',                        # Flat files string (auto-set)
    'database': '',                      # Database path (auto-set)
    'k': 12,                             # Stars for plate solving
    'm': 30,                             # Stars for fitting stack
    'n': 30,                             # Stars for verifying stack
    'double_star_cutoff': 10,            # Double star cutoff (arcseconds)
    'double_star_mag': 17,               # Maximum magnitude for double stars
    'DEFAULT_DATE': '2020-01-01',       # Default date for date guessing
    'do_tetra_platesolve': False,        # Use tetra platesolve (legacy)
    'basis_type': 'polynomial',          # Basis type: 'polynomial' or 'legendre'
    'flag_display2': True,               # Show graphics in Step 2
    'flag_display3': True,               # Show graphics in Step 3
    'flag_debug': False,                 # Debug mode
    'output_dir': output_directory,      # Output directory
}

print("✅ Configuration loaded!")
print(f"   Light files: {len(light_files)}")
print(f"   Dark files: {len(dark_files)}")
print(f"   Flat files: {len(flat_files)}")
print(f"   Output directory: {output_directory if output_directory else '(same as input)'}")

### Validate Input Files

Before processing, let's check that all your input files exist and can be opened.

**✅ This validation step:**
- Checks that all file paths are correct
- Verifies files exist and can be read
- Reports any missing or invalid files
- Must pass before proceeding to processing

In [ ]:
# ============================================================================
# FILE VALIDATION: Check that all input files exist and are readable
# ============================================================================
# This function checks each file to make sure:
#   1. The file exists
#   2. The file can be opened (not corrupted, permissions OK)
#   3. Returns a list of valid files
# ============================================================================

def validate_files(file_list, file_type="light"):
    """
    Check if files exist and can be opened.
    
    Parameters:
    -----------
    file_list : list
        List of file paths to check
    file_type : str
        Type of files (for error messages): 'light', 'dark', or 'flat'
    
    Returns:
    --------
    valid_files : list
        List of valid file paths
    """
    valid_files = []
    for file_path in file_list:
        if not file_path:  # Skip empty strings
            continue
        if not os.path.exists(file_path):
            print(f"⚠️  WARNING: {file_type} file not found: {file_path}")
            continue
        try:
            # Try to open the file to verify it's readable
            with open(file_path, 'rb') as f:
                pass  # Just check if we can open it
            valid_files.append(file_path)
            print(f"✅ Valid {file_type} file: {os.path.basename(file_path)}")
        except Exception as e:
            print(f"❌ ERROR: Cannot open {file_type} file {file_path}: {e}")
    return valid_files

# ============================================================================
# VALIDATE ALL INPUT FILES
# ============================================================================

print("=" * 60)
print("VALIDATING INPUT FILES")
print("=" * 60)
print()

# Validate each type of file
valid_light_files = validate_files(light_files, "light")
valid_dark_files = validate_files(dark_files, "dark")
valid_flat_files = validate_files(flat_files, "flat")

# Summary
print()
print("=" * 60)
print("VALIDATION SUMMARY")
print("=" * 60)
print(f"   Light files: {len(valid_light_files)} valid out of {len(light_files)} specified")
print(f"   Dark files:  {len(valid_dark_files)} valid out of {len(dark_files)} specified")
print(f"   Flat files:  {len(valid_flat_files)} valid out of {len(flat_files)} specified")
print()

# Check if we have at least one light file (required!)
if len(valid_light_files) == 0:
    print("❌ ERROR: No valid light files found!")
    print("   Please check your file paths in the configuration cell above.")
    print("   Make sure to use absolute paths or paths relative to this notebook.")
    raise ValueError("No valid light files provided. Cannot proceed without light images.")
else:
    print("✅ Validation complete! Ready to process images.")
    print(f"   Processing {len(valid_light_files)} light frame(s)...")

### Process Images: Stack and Find Stars

This is where the actual processing happens. This may take several minutes depending on:
- Number of images
- Image size
- Number of stars in the field

In [ ]:
# ============================================================================
# STEP 1: STACK IMAGES AND FIND STAR CENTROIDS
# ============================================================================
# 
# This function does the following:
#   1. Opens all your light images (and optional dark/flat frames)
#   2. Aligns them using star patterns (matches stars across frames)
#   3. Stacks them together (averages them) to reduce noise
#   4. Finds star positions (centroids) in the stacked image
#   5. Does "plate solving" - matches stars to catalog to find where you're pointing
#
# Function called: stacker_implementation.do_stack()
#   - This is the same function used by GUI Tab 1
#   - Takes: light files, dark files, flat files, options dictionary
#   - Returns: Nothing (saves output files directly)
#
# Output: A ZIP file containing:
#   - Star positions (centroids) in pixel coordinates
#   - Plate solution (RA, Dec, roll, plate scale in arcsec/pixel)
#   - Stacked image
#   - Log file with processing details
#
# This output file will be used as input for Step 2.
# ============================================================================

print("=" * 60)
print("STARTING IMAGE STACKING AND CENTROID DETECTION")
print("=" * 60)
print()
print(f"Processing {len(valid_light_files)} light frame(s)...")
if valid_dark_files:
    print(f"Using {len(valid_dark_files)} dark frame(s) for noise reduction")
if valid_flat_files:
    print(f"Using {len(valid_flat_files)} flat frame(s) for vignetting correction")
print()
print("This may take several minutes depending on:")
print("  - Number of images")
print("  - Image size")
print("  - Number of stars in the field")
print()
print("Starting processing...")
print()

try:
    # Call the stacking function (same as GUI Tab 1)
    # This is the main processing step - it does all the work!
    stacker_implementation.do_stack(
        valid_light_files,    # Your light images (required)
        valid_dark_files,     # Dark frames (optional, for noise reduction)
        valid_flat_files,     # Flat frames (optional, for vignetting correction)
        options               # All the settings from configuration
    )
    
    print()
    print("=" * 60)
    print("✅ SUCCESS! Image stacking complete!")
    print("=" * 60)
    print()
    print("📁 Check your output directory for the results:")
    if output_directory:
        print(f"   Output directory: {output_directory}")
    else:
        print(f"   Output directory: {os.path.dirname(valid_light_files[0])}")
    print()
    print("   Look for files named:")
    print("   - centroid_data{timestamp}.zip  (main output file)")
    print("   - CENTROID_OUTPUT{timestamp}/    (output folder)")
    print()
    print("➡️  The ZIP file will be used as input for Step 2 (Compute Distortion)")
    
except Exception as e:
    print()
    print("=" * 60)
    print("❌ ERROR during stacking!")
    print("=" * 60)
    print(f"Error message: {e}")
    print()
    print("Common issues:")
    print("  - File paths incorrect (check configuration cell)")
    print("  - Images corrupted or wrong format")
    print("  - Not enough stars in images for plate solving")
    print("  - Images too dark or too bright")
    print()
    raise  # Re-raise the error so you can see the full traceback

### View Results

Let's check what was created and find the output file for use in Step 2.

In [ ]:
# ============================================================================
# DISPLAY RESULTS: Show what was created
# ============================================================================
# This cell finds the output files created by Step 1 and displays information
# about them. The most recent output file will be used automatically in Step 2.
# ============================================================================

import glob
from pathlib import Path

# Determine output directory
if output_directory and os.path.isdir(output_directory):
    output_path = Path(output_directory)
else:
    # Use same directory as first input file
    output_path = Path(valid_light_files[0]).parent

print("=" * 60)
print("STEP 1 OUTPUT FILES")
print("=" * 60)
print()
print(f"Searching in: {output_path}")
print()

# Find centroid ZIP files (sorted by modification time, newest first)
centroid_zips = sorted(
    output_path.glob("centroid_data*.zip"), 
    key=lambda x: x.stat().st_mtime, 
    reverse=True
)

# Find output directories
output_dirs = sorted(
    output_path.glob("CENTROID_OUTPUT*"), 
    key=lambda x: x.stat().st_mtime if x.is_dir() else 0, 
    reverse=True
)

if centroid_zips:
    latest_zip = centroid_zips[0]
    print("✅ Found output files!")
    print()
    print(f"📦 Main output file (for Step 2):")
    print(f"   Name: {latest_zip.name}")
    print(f"   Full path: {latest_zip}")
    print(f"   Size: {latest_zip.stat().st_size / (1024*1024):.2f} MB")
    print()
    print("📝 This ZIP file contains:")
    print("   - Star positions (centroids) in pixel coordinates")
    print("   - Plate solution (RA, Dec, roll, plate scale)")
    print("   - Stacked image data")
    print("   - Processing log")
    print()
    
    # Store for use in Step 2
    step1_output_file = str(latest_zip)
    print(f"💾 Saved as 'step1_output_file' for use in Step 2")
    print()
    
    if output_dirs:
        print(f"📁 Output directory: {output_dirs[0].name}")
        print(f"   Contains plots, logs, and intermediate files")
        print()
    
    print("=" * 60)
    print("✅ Step 1 Complete! Ready for Step 2.")
    print("=" * 60)
    print()
    print("➡️  Next: Go to Step 2 (Compute Distortion) section below")
    print("   The output file path will be used automatically")
    
else:
    print("⚠️  No output files found!")
    print()
    print("This could mean:")
    print("  - Processing is still running (check above for progress)")
    print("  - An error occurred (check error messages above)")
    print("  - Output directory is different than expected")
    print()
    print(f"   Searched in: {output_path}")
    print("   Looking for files matching: centroid_data*.zip")
    print()
    print("💡 Tip: If processing completed successfully, the output file")
    print("   should be in the same directory as your input images.")

### Configuration: Set Input File and Options for Distortion Fitting

**IMPORTANT:** You need the output ZIP file from Step 1 to proceed!

In [ ]:
# ============================================================================
# CONFIGURATION: Set input file and options for distortion fitting
# ============================================================================
# 
# INSTRUCTIONS:
# 1. The input file should be the ZIP file created by Step 1
# 2. If you ran Step 1 above, the file path will be set automatically
# 3. Otherwise, manually set 'centroid_zip_path' to your Step 1 output file
# ============================================================================

# Path to the centroid ZIP file from Step 1
# This should be the file named: centroid_data{timestamp}.zip
centroid_zip_path = ''

# If Step 1 was run above, use the latest output automatically
if 'step1_output_file' in locals():
    centroid_zip_path = step1_output_file
    print(f"✅ Using Step 1 output: {os.path.basename(centroid_zip_path)}")
elif 'latest_zip' in locals():
    centroid_zip_path = str(latest_zip)
    print(f"✅ Using latest Step 1 output: {os.path.basename(centroid_zip_path)}")
else:
    print("⚠️  No Step 1 output found. Please set 'centroid_zip_path' manually.")
    print("   Example: centroid_zip_path = '/path/to/centroid_data20240101120000.zip'")

# ============================================================================
# DISTORTION FITTING OPTIONS: Adjust these as needed
# ============================================================================

# Star catalog and matching options
options['max_star_mag_dist'] = 12          # Maximum star magnitude to use for distortion fitting
options['rough_match_threshhold'] = 36     # Rough match threshold (arcseconds)
options['safety_limit_mag'] = 13           # Safety limit for star magnitude

# Distortion polynomial options
options['distortionOrder'] = 'cubic'        # Polynomial order: 'linear', 'cubic', 'quintic', 'septic'
                                            # Higher order = more flexible but needs more stars
options['distortion_fit_tol'] = 1.0        # Fit tolerance (arcseconds) - how accurate the fit should be

# Date and location options (needed if using corrections)
options['observation_date'] = '2023-12-01'  # Observation date (YYYY-MM-DD format)
options['guess_date'] = False              # Auto-guess observation date from star positions

# Correction options (advanced - usually leave disabled for basic use)
options['enable_corrections'] = False       # Enable aberration and parallax corrections
options['enable_corrections_ref'] = False   # Enable atmospheric refraction correction
options['enable_gravitational_def'] = False # Enable gravitational deflection correction
                                            # (Usually disabled here, enabled in Step 3)

# Observation parameters (only needed if corrections are enabled)
options['observation_time'] = ''            # Observation time (HH:MM:SS UTC)
options['observation_lat'] = ''              # Observation latitude (degrees)
options['observation_long'] = ''            # Observation longitude (degrees)
options['observation_temp'] = 10            # Temperature (°C) for refraction
options['observation_pressure'] = 1010      # Pressure (millibars) for refraction
options['observation_humidity'] = 0         # Humidity (0.0 to 1.0) for refraction
options['observation_height'] = 0            # Height above sea level (meters)
options['observation_wavelength'] = 0.65     # Wavelength (micrometers) for refraction

# Advanced distortion options
options['distortion_reference_files'] = ''   # Reference distortion files (optional)
options['distortion_fixed_coefficients'] = 'None'  # Fix certain coefficients: 'None', 'constant', 'linear', etc.
options['crop_circle'] = False              # Crop to circular region
options['crop_circle_thresh'] = 1.0         # Crop circle threshold
options['remove_double_tab2'] = False       # Remove double stars

# Display options
options['flag_display2'] = True              # Show graphics during processing

# ============================================================================
# VALIDATE INPUT FILE
# ============================================================================

if not centroid_zip_path:
    print("❌ ERROR: No input file specified!")
    print("   Please set 'centroid_zip_path' to your Step 1 output file.")
    raise ValueError("centroid_zip_path is required")

if not os.path.exists(centroid_zip_path):
    print(f"❌ ERROR: File not found: {centroid_zip_path}")
    print("   Please check the file path.")
    raise FileNotFoundError(f"Input file not found: {centroid_zip_path}")

# Verify it's a ZIP file
if not centroid_zip_path.endswith('.zip'):
    print(f"⚠️  WARNING: File doesn't end with .zip: {centroid_zip_path}")
    print("   Make sure this is the correct file from Step 1")

print()
print("=" * 60)
print("STEP 2 CONFIGURATION")
print("=" * 60)
print(f"✅ Input file: {os.path.basename(centroid_zip_path)}")
print(f"   Full path: {centroid_zip_path}")
print(f"✅ Distortion order: {options['distortionOrder']}")
print(f"✅ Fit tolerance: {options['distortion_fit_tol']} arcseconds")
print(f"✅ Max star magnitude: {options['max_star_mag_dist']}")
print()
print("Ready to process!")

### Run Distortion Fitting

This step matches stars to the Gaia catalog and fits the optical distortion model.

In [ ]:
# ============================================================================
# STEP 2: COMPUTE DISTORTION MODEL
# ============================================================================
# 
# This function does the following:
#   1. Opens the centroid data ZIP file from Step 1
#   2. Does plate solving (refines where you're pointing in the sky)
#   3. Queries Gaia catalog to identify which stars are which
#   4. Gets precise star positions from Gaia (with proper motions, parallaxes)
#   5. Compares measured positions vs. Gaia positions
#   6. Fits a polynomial to correct for optical distortion
#
# Function called: distortion_fitter.match_and_fit_distortion()
#   - This is the same function used by GUI Tab 2
#   - Takes: ZIP file path, options dictionary, debug folder (optional)
#   - Returns: Nothing (saves output files directly)
#
# Output: A ZIP file containing:
#   - Distortion polynomial coefficients (how to correct for lens/telescope distortion)
#   - Matched stars with errors (which stars matched, how accurate)
#   - Refined plate solution (improved RA, Dec, roll, plate scale)
#
# This output file will be used as input for Step 3.
# ============================================================================

print("=" * 60)
print("STARTING DISTORTION FITTING")
print("=" * 60)
print()
print(f"Input file: {os.path.basename(centroid_zip_path)}")
print()
print("This step will:")
print("  1. Load star positions from Step 1")
print("  2. Match stars to Gaia catalog (may take a few minutes)")
print("  3. Fit distortion polynomial")
print()
print("Starting processing...")
print()

try:
    # Call the distortion fitting function (same as GUI Tab 2)
    # This is the main processing step for Step 2
    distortion_fitter.match_and_fit_distortion(
        centroid_zip_path,  # Input: ZIP file from Step 1
        options,             # All the settings from configuration
        None                 # Debug folder (optional, set to None)
    )
    
    print()
    print("=" * 60)
    print("✅ SUCCESS! Distortion fitting complete!")
    print("=" * 60)
    print()
    print("📁 Check your output directory for the results:")
    if output_directory:
        print(f"   Output directory: {output_directory}")
    else:
        # Try to determine from input file location
        input_dir = os.path.dirname(centroid_zip_path)
        print(f"   Output directory: {input_dir}")
    print()
    print("   Look for files named:")
    print("   - distortion_data{timestamp}.zip  (main output file)")
    print("   - DISTORTION_OUTPUT{timestamp}/   (output folder)")
    print()
    print("➡️  The ZIP file will be used as input for Step 3 (Eclipse Analysis)")
    
except Exception as e:
    print()
    print("=" * 60)
    print("❌ ERROR during distortion fitting!")
    print("=" * 60)
    print(f"Error message: {e}")
    print()
    print("Common issues:")
    print("  - Step 1 output file is invalid or corrupted")
    print("  - Not enough stars matched to Gaia catalog")
    print("  - Internet connection needed for Gaia queries")
    print("  - Observation date incorrect (affects star positions)")
    print("  - Plate solving failed in Step 1")
    print()
    raise  # Re-raise the error so you can see the full traceback

### View Distortion Results

Let's check what was created and find the output file for use in Step 3.

In [ ]:
# ============================================================================
# DISPLAY DISTORTION RESULTS: Show what was created
# ============================================================================
# This cell finds the output files created by Step 2 and displays information
# about them. The most recent output file will be used automatically in Step 3.
# ============================================================================

# Determine output directory (same logic as Step 1)
if output_directory and os.path.isdir(output_directory):
    output_path = Path(output_directory)
else:
    # Use same directory as input file
    output_path = Path(centroid_zip_path).parent

print("=" * 60)
print("STEP 2 OUTPUT FILES")
print("=" * 60)
print()
print(f"Searching in: {output_path}")
print()

# Find distortion ZIP files (sorted by modification time, newest first)
distortion_zips = sorted(
    output_path.glob("distortion_data*.zip"), 
    key=lambda x: x.stat().st_mtime, 
    reverse=True
)

# Find output directories
output_dirs = sorted(
    output_path.glob("DISTORTION_OUTPUT*"), 
    key=lambda x: x.stat().st_mtime if x.is_dir() else 0, 
    reverse=True
)

if distortion_zips:
    latest_distortion_zip = distortion_zips[0]
    print("✅ Found output files!")
    print()
    print(f"📦 Main output file (for Step 3):")
    print(f"   Name: {latest_distortion_zip.name}")
    print(f"   Full path: {latest_distortion_zip}")
    print(f"   Size: {latest_distortion_zip.stat().st_size / (1024*1024):.2f} MB")
    print()
    print("📝 This ZIP file contains:")
    print("   - Distortion polynomial coefficients")
    print("   - Matched stars with position errors")
    print("   - Refined plate solution")
    print("   - Processing diagnostics")
    print()
    
    # Store for use in Step 3
    step2_output_file = str(latest_distortion_zip)
    print(f"💾 Saved as 'step2_output_file' for use in Step 3")
    print()
    
    if output_dirs:
        print(f"📁 Output directory: {output_dirs[0].name}")
        print(f"   Contains plots, logs, and intermediate files")
        print()
    
    # Try to peek inside the ZIP to show what's there
    try:
        with zipfile.ZipFile(latest_distortion_zip, 'r') as z:
            file_list = z.namelist()
            print("📋 Files inside ZIP:")
            for f in file_list[:10]:  # Show first 10 files
                print(f"   - {f}")
            if len(file_list) > 10:
                print(f"   ... and {len(file_list) - 10} more files")
    except:
        pass  # If we can't read it, that's okay
    
    print()
    print("=" * 60)
    print("✅ Step 2 Complete! Ready for Step 3.")
    print("=" * 60)
    print()
    print("➡️  Next: Go to Step 3 (Eclipse Analysis) section below")
    print("   The output file path will be used automatically")
    
else:
    print("⚠️  No output files found!")
    print()
    print("This could mean:")
    print("  - Processing is still running (check above for progress)")
    print("  - An error occurred (check error messages above)")
    print("  - Output directory is different than expected")
    print()
    print(f"   Searched in: {output_path}")
    print("   Looking for files matching: distortion_data*.zip")
    print()
    print("💡 Tip: If processing completed successfully, the output file")
    print("   should be in the same directory as your Step 1 output.")

---

# 📍 Step 1: Find Centroids (Image Stacking)

**This section corresponds to Tab 1 in the original GUI.**

---

## 🎯 What This Step Does

This is the **first phase** of the analysis pipeline. It processes your raw eclipse images and identifies star positions.

### The Process:

1. **Opens your images** - Reads all the light frames (and optional dark/flat frames for calibration)
2. **Aligns images** - Matches star patterns across frames to align them precisely
3. **Stacks images** - Averages all aligned images to reduce noise and improve signal
4. **Finds stars** - Detects star positions (centroids) in the stacked image using pattern recognition
5. **Plate solves** - Matches detected stars to a star catalog to determine:
   - Where you're pointing in the sky (RA, Dec)
   - Image orientation (roll angle)
   - Plate scale (arcseconds per pixel)

### Why This Step is Needed:

- **Image stacking** reduces noise by averaging multiple frames
- **Centroid detection** finds precise star positions (needed for later analysis)
- **Plate solving** establishes the celestial coordinate system (needed to compare with star catalogs)

### Inputs Required:

- **Light frames** (required): Your eclipse images (FITS format)
- **Dark frames** (optional): For noise reduction
- **Flat frames** (optional): For vignetting correction

### Output:

A ZIP file named `centroid_data{timestamp}.zip` containing:
- **Star positions** (centroids) in pixel coordinates
- **Plate solution** (RA, Dec, roll, plate scale)
- **Stacked image** (FITS format)
- **Processing metadata** (JSON format)

**This output file will be used as input for Step 2.**

### What to Expect:

- Processing time: **5-15 minutes** depending on number of images
- Output file size: **10-50 MB** typically
- Number of stars found: **50-200** typically (depends on your images)

### Common Issues:

- **"No stars found"** → Images may be too dark/bright, or Sun/Moon blocking stars
- **"Plate solving failed"** → Not enough stars detected, or images don't overlap properly
- **"File not found"** → Check that file paths are correct and files exist

---

---

# 🔧 Step 2: Compute Distortion (Plate Solving & Distortion Fitting)

**This section corresponds to Tab 2 in the original GUI.**

---

## 🎯 What This Step Does

This is the **second phase** of the analysis pipeline. It refines star positions and corrects for optical distortion in your telescope/camera system.

### The Process:

1. **Loads Step 1 data** - Opens the centroid ZIP file from Step 1
2. **Refines plate solution** - Improves the initial plate solve from Step 1 for better accuracy
3. **Matches to Gaia catalog** - Identifies which detected stars correspond to which catalog stars
   - Uses the **Gaia catalog** (most precise star positions available)
   - Queries online database (internet connection required)
4. **Gets precise positions** - Retrieves accurate star positions from Gaia, including:
   - Proper motions (how stars move over time)
   - Parallaxes (distance corrections)
   - Updated positions for your observation date
5. **Fits distortion model** - Calculates how much your telescope/lens distorts the image
   - Fits a polynomial to correct for optical aberrations
   - Accounts for lens distortion, field curvature, etc.

### Why This Step is Needed:

- **Gaia catalog matching** provides the most accurate reference positions (better than Tycho-2)
- **Distortion correction** is essential - all real telescopes/lenses have some distortion
- **Proper motion correction** accounts for stars moving over time (important for precise measurements)
- **Without this step**, you can't accurately measure small deflections (like gravitational deflection)

### Inputs Required:

- **Step 1 output ZIP file** (required): `centroid_data{timestamp}.zip`
- **Observation date** (required): When the images were taken (YYYY-MM-DD format)
- **Observation location** (optional): Latitude/longitude if using corrections

### Output:

A ZIP file named `distortion_data{timestamp}.zip` containing:
- **Distortion polynomial coefficients** (how to correct for optical distortion)
- **Matched stars with position errors** (which stars matched, how accurate)
- **Refined plate solution** (improved RA, Dec, roll, plate scale)
- **Processing diagnostics** (JSON format)

**This output file will be used as input for Step 3.**

### What to Expect:

- Processing time: **10-30 minutes** (depends on number of stars and internet speed)
- Output file size: **5-20 MB** typically
- Number of matched stars: **20-100** typically (depends on field of view and star density)
- Distortion fit quality: **RMS error < 1 arcsecond** is good

### Common Issues:

- **"No stars matched to Gaia"** → Check observation date is correct, or field has too few bright stars
- **"Internet connection error"** → Gaia queries require internet - check your connection
- **"Distortion fit failed"** → Not enough matched stars, or stars too clustered
- **"Plate solving failed"** → Step 1 may have had issues - check Step 1 output

---

---

# 🌟 Step 3: Eclipse Analysis (Calculate Deflection Coefficient)

**This section corresponds to Tab 3 in the original GUI.**

---

## 🎯 What This Step Does

This is the **final phase** of the analysis pipeline. It calculates Einstein's gravitational deflection coefficient by measuring how starlight was bent by the Sun's gravity.

### The Process:

1. **Loads Step 2 data** - Opens the distortion ZIP file from Step 2
2. **Applies corrections** - Uses the distortion model from Step 2 to correct star positions
   - Removes optical distortion effects
   - Applies atmospheric refraction corrections (if enabled)
   - Accounts for proper motion and parallax
3. **Calculates angular distances** - Measures how far each star is from the Sun (in arcseconds)
4. **Calculates deflections** - Compares observed star positions to expected positions
   - Measures how much each star was deflected by the Sun's gravity
   - Deflection is largest for stars near the Sun's limb
5. **Fits deflection model** - Finds the best-fit value for Einstein's deflection coefficient **L**
   - Uses the formula: deflection = L × (solar radius / angular distance)
   - Einstein's prediction: **L = 1.751 arcseconds**

### Why This Step is Needed:

- **This is the scientific goal** - measuring gravitational light deflection
- **Tests Einstein's theory** - compares measured L to predicted value (1.751 arcsec)
- **Requires all previous steps** - needs accurate star positions and distortion corrections

### Inputs Required:

- **Step 2 output ZIP file** (required): `distortion_data{timestamp}.zip`
- **Observation date and time** (required): When eclipse occurred (for Sun position)
- **Observation location** (required): Latitude/longitude (for Sun position calculation)

### Output:

Final results including:
- **Deflection coefficient L** (arcseconds) - should be ~1.751
- **Uncertainty/error** in the measurement
- **Number of stars used** in the analysis
- **Diagnostic plots** showing the fit
- **Text file** (`ECLIPSE_OUTPUT{timestamp}.txt`) with detailed results

### What to Expect:

- Processing time: **5-15 minutes**
- Expected result: **L ≈ 1.751 ± 0.1 arcseconds** (Einstein's prediction)
- Number of stars used: **10-50** typically (stars near the Sun)
- Quality indicators:
  - **Good fit:** χ² close to 1, small uncertainty
  - **Poor fit:** Large scatter, high uncertainty

### Common Issues:

- **"L value very different from 1.751"** → Check observation date/time, or systematic errors in earlier steps
- **"Not enough stars"** → Need stars close to Sun (within ~10 solar radii)
- **"Large uncertainty"** → May need more stars, or check data quality
- **"Sun position incorrect"** → Verify observation date, time, and location are correct

---

### Configuration: Set Input File and Options for Eclipse Analysis

**IMPORTANT:** You need the output ZIP file from Step 2 to proceed!

In [ ]:
# ============================================================================
# CONFIGURATION: Set input file and options for eclipse analysis
# ============================================================================
# 
# INSTRUCTIONS:
# 1. The input file should be the ZIP file created by Step 2
# 2. If you ran Step 2 above, the file path will be set automatically
# 3. Otherwise, manually set 'distortion_zip_path' to your Step 2 output file
# ============================================================================

# Path to the distortion ZIP file from Step 2
# This should be the file named: distortion_data{timestamp}.zip
distortion_zip_path = ''

# If Step 2 was run above, use the latest output automatically
if 'step2_output_file' in locals():
    distortion_zip_path = step2_output_file
    print(f"✅ Using Step 2 output: {os.path.basename(distortion_zip_path)}")
elif 'latest_distortion_zip' in locals():
    distortion_zip_path = str(latest_distortion_zip)
    print(f"✅ Using latest Step 2 output: {os.path.basename(distortion_zip_path)}")
else:
    print("⚠️  No Step 2 output found. Please set 'distortion_zip_path' manually.")
    print("   Example: distortion_zip_path = '/path/to/distortion_data20240101120000.zip'")

# ============================================================================
# ECLIPSE ANALYSIS OPTIONS: Adjust these as needed
# ============================================================================

# Analysis method: How to calculate the deflection coefficient
# Options: 'Method 1', 'Method 2', 'Method 1 & 2'
#   - Method 1: Direct radial fit
#   - Method 2: Alternative fitting approach
#   - Method 1 & 2: Use both methods and compare
options['eclipse_method'] = 'Method 1 & 2'  # Recommended: use both methods

# Star selection options
options['eclipse_limiting_mag'] = 11         # Maximum star magnitude to use (brighter = better)
                                             # Lower number = brighter stars only
options['remove_double_stars_eclipse'] = False  # Remove double/binary stars from analysis

# Radial cutoff (optional - limits analysis to stars within certain distance from Sun)
options['limit_radial_sun_radii'] = False    # Enable radial cutoff
options['limit_radial_sun_radii_value'] = 9   # Cutoff distance in solar radii
                                             # Only used if limit_radial_sun_radii = True

# Center object: What to use as the reference center
options['object_centre_moon'] = False         # Center on Moon instead of Sun
                                             # Usually False (center on Sun)

# Display options
options['flag_display3'] = True              # Show graphics during processing

# ============================================================================
# VALIDATE INPUT FILE
# ============================================================================

if not distortion_zip_path:
    print("❌ ERROR: No input file specified!")
    print("   Please set 'distortion_zip_path' to your Step 2 output file.")
    raise ValueError("distortion_zip_path is required")

if not os.path.exists(distortion_zip_path):
    print(f"❌ ERROR: File not found: {distortion_zip_path}")
    print("   Please check the file path.")
    raise FileNotFoundError(f"Input file not found: {distortion_zip_path}")

# Verify it's a ZIP file
if not distortion_zip_path.endswith('.zip'):
    print(f"⚠️  WARNING: File doesn't end with .zip: {distortion_zip_path}")
    print("   Make sure this is the correct file from Step 2")

print()
print("=" * 60)
print("STEP 3 CONFIGURATION")
print("=" * 60)
print(f"✅ Input file: {os.path.basename(distortion_zip_path)}")
print(f"   Full path: {distortion_zip_path}")
print(f"✅ Analysis method: {options['eclipse_method']}")
print(f"✅ Limiting magnitude: {options['eclipse_limiting_mag']}")
print(f"✅ Remove double stars: {options['remove_double_stars_eclipse']}")
print(f"✅ Center on Moon: {options['object_centre_moon']}")
if options['limit_radial_sun_radii']:
    print(f"✅ Radial cutoff: {options['limit_radial_sun_radii_value']} solar radii")
else:
    print(f"✅ Radial cutoff: Disabled (using all stars)")
print()
print("Ready to process!")

### Run Eclipse Analysis

This step calculates the gravitational deflection coefficient (Einstein's L).

In [ ]:
# ============================================================================
# STEP 3: ECLIPSE ANALYSIS - Calculate Deflection Coefficient
# ============================================================================
# 
# This function does the following:
#   1. Opens the distortion data ZIP file from Step 2
#   2. Loads star positions (observed and catalog positions)
#   3. Calculates the angular distance of each star from the Sun
#   4. Applies corrections (distortion, refraction, etc.)
#   5. Measures how much each star was deflected by the Sun's gravity
#   6. Fits a model to find Einstein's deflection coefficient L
#
# Function called: eclipse_analysis.eclipse_analysis()
#   - This is the same function used by GUI Tab 3
#   - Takes: ZIP file path, options dictionary
#   - Returns: Nothing (saves output files directly)
#
# Output: Final results including:
#   - Deflection coefficient L (should be ~1.751 arcseconds for Einstein's prediction)
#   - Uncertainty in the measurement
#   - Diagnostic plots showing the fit
#   - Text file with detailed results
#
# The deflection coefficient L tells us how much starlight is bent by gravity.
# Einstein's theory predicts L = 1.751 arcseconds at the limb of the Sun.
# ============================================================================

print("=" * 60)
print("STARTING ECLIPSE ANALYSIS")
print("=" * 60)
print()
print(f"Input file: {os.path.basename(distortion_zip_path)}")
print()
print("This step will:")
print("  1. Load star positions from Step 2")
print("  2. Calculate angular distances from the Sun")
print("  3. Measure gravitational deflection for each star")
print("  4. Fit deflection model to find coefficient L")
print()
print("Expected result: L ≈ 1.751 arcseconds (Einstein's prediction)")
print()
print("Starting processing...")
print()

try:
    # Call the eclipse analysis function (same as GUI Tab 3)
    # This is the main processing step for Step 3
    eclipse_analysis.eclipse_analysis(
        distortion_zip_path,  # Input: ZIP file from Step 2
        options                # All the settings from configuration
    )
    
    print()
    print("=" * 60)
    print("✅ SUCCESS! Eclipse analysis complete!")
    print("=" * 60)
    print()
    print("📁 Check your output directory for the results:")
    if output_directory:
        print(f"   Output directory: {output_directory}")
    else:
        # Try to determine from input file location
        input_dir = os.path.dirname(distortion_zip_path)
        print(f"   Output directory: {input_dir}")
    print()
    print("   Look for files named:")
    print("   - ECLIPSE_OUTPUT{timestamp}.txt  (final results with L value)")
    print("   - ECLIPSE_OUTPUT{timestamp}/     (output folder with plots)")
    print()
    print("📊 The results file contains:")
    print("   - Deflection coefficient L (arcseconds)")
    print("   - Uncertainty/error in the measurement")
    print("   - Number of stars used")
    print("   - Diagnostic information")
    print()
    print("🎉 Analysis complete! Check the output file for your results.")
    
except Exception as e:
    print()
    print("=" * 60)
    print("❌ ERROR during eclipse analysis!")
    print("=" * 60)
    print(f"Error message: {e}")
    print()
    print("Common issues:")
    print("  - Step 2 output file is invalid or corrupted")
    print("  - Not enough stars in the analysis")
    print("  - Observation date/time incorrect (affects Sun/Moon positions)")
    print("  - Stars too far from Sun (need stars near the Sun for deflection)")
    print("  - Missing observation location (lat/long needed for Sun position)")
    print()
    raise  # Re-raise the error so you can see the full traceback

### View Final Results

Let's check the results and see the deflection coefficient L that was calculated.

In [ ]:
 ============================================================================
# DISPLAY ECLIPSE ANALYSIS RESULTS: Show final deflection coefficient
# ============================================================================
# This cell finds the output files created by Step 3 and displays the
# final deflection coefficient L and other key results.
# ============================================================================

# Determine output directory (same logic as previous steps)
if output_directory and os.path.isdir(output_directory):
    output_path = Path(output_directory)
else:
    # Use same directory as input file
    output_path = Path(distortion_zip_path).parent

print("=" * 60)
print("STEP 3 OUTPUT FILES")
print("=" * 60)
print()
print(f"Searching in: {output_path}")
print()

# Find eclipse output text files (sorted by modification time, newest first)
eclipse_outputs = sorted(
    output_path.glob("ECLIPSE_OUTPUT*.txt"), 
    key=lambda x: x.stat().st_mtime, 
    reverse=True
)

# Find output directories
output_dirs = sorted(
    output_path.glob("ECLIPSE_OUTPUT*"), 
    key=lambda x: x.stat().st_mtime if x.is_dir() else 0, 
    reverse=True
)

if eclipse_outputs:
    latest_output = eclipse_outputs[0]
    print("✅ Found output files!")
    print()
    print(f"📄 Main results file:")
    print(f"   Name: {latest_output.name}")
    print(f"   Full path: {latest_output}")
    print()
    
    # Read and display the results file
    try:
        with open(latest_output, 'r') as f:
            results_content = f.read()
        
        print("=" * 60)
        print("FINAL RESULTS")
        print("=" * 60)
        print()
        print(results_content)
        print()
        print("=" * 60)
        
        # Try to extract key values (L coefficient) from the text
        # The format may vary, but we'll look for common patterns
        lines = results_content.split('\n')
        l_value = None
        uncertainty = None
        
        for line in lines:
            # Look for deflection coefficient
            if 'deflection' in line.lower() and 'constant' in line.lower():
                # Try to extract number
                import re
                numbers = re.findall(r'[-+]?\d*\.\d+|\d+', line)
                if numbers:
                    l_value = float(numbers[0])
            if 'error' in line.lower() or 'uncertainty' in line.lower() or 'std' in line.lower():
                # Try to extract number
                import re
                numbers = re.findall(r'[-+]?\d*\.\d+|\d+', line)
                if numbers:
                    uncertainty = float(numbers[0])
        
        if l_value is not None:
            print()
            print("🎯 KEY RESULT:")
            print(f"   Deflection Coefficient L = {l_value:.4f} arcseconds")
            if uncertainty is not None:
                print(f"   Uncertainty = ±{uncertainty:.4f} arcseconds")
            print()
            print(f"   Einstein's prediction: L = 1.751 arcseconds")
            if l_value is not None:
                difference = abs(l_value - 1.751)
                print(f"   Difference from prediction: {difference:.4f} arcseconds")
            print()
        
    except Exception as e:
        print(f"⚠️  Could not read results file: {e}")
        print("   Please open the file manually to view results")
        print()
    
    if output_dirs:
        print(f"📁 Output directory: {output_dirs[0].name}")
        print(f"   Contains plots, diagnostic files, and detailed analysis")
        print()
        
        # List some files in the output directory
        try:
            dir_path = output_dirs[0]
            files_in_dir = list(dir_path.glob("*"))
            if files_in_dir:
                print("📋 Files in output directory:")
                for f in files_in_dir[:10]:  # Show first 10 files
                    if f.is_file():
                        print(f"   - {f.name}")
                if len([f for f in files_in_dir if f.is_file()]) > 10:
                    print(f"   ... and more files")
        except:
            pass
    
    print()
    print("=" * 60)
    print("✅ Step 3 Complete! Analysis finished!")
    print("=" * 60)
    print()
    print("📊 Summary:")
    print("   - Deflection coefficient L has been calculated")
    print("   - Results saved to output file")
    print("   - Diagnostic plots available in output directory")
    print()
    print("💡 Next steps:")
    print("   - Review the results file for detailed analysis")
    print("   - Check plots in the output directory")
    print("   - Compare your L value to Einstein's prediction (1.751 arcsec)")
    print()
    print("🎉 Congratulations! You've completed the full MEE2024 analysis!")
    
else:
    print("⚠️  No output files found!")
    print()
    print("This could mean:")
    print("  - Processing is still running (check above for progress)")
    print("  - An error occurred (check error messages above)")
    print("  - Output directory is different than expected")
    print()
    print(f"   Searched in: {output_path}")
    print("   Looking for files matching: ECLIPSE_OUTPUT*.txt")
    print()
    print("💡 Tip: If processing completed successfully, the output file")
    print("   should be in the same directory as your Step 2 output.")

---

# 🔧 Troubleshooting Guide

This section helps you diagnose and fix common issues that may arise during analysis.

---

## Common Issues and Solutions

### 1. File Not Found Errors

**Symptoms:**
- Error messages like `FileNotFoundError` or `No such file or directory`
- Files not being detected during validation

**Solutions:**
- ✅ **Check file paths are correct** - Use absolute paths (full path from root) for best results
- ✅ **Verify files exist** - Use file explorer to confirm files are where you think they are
- ✅ **Check file permissions** - Make sure files are readable (not locked by another program)
- ✅ **Use forward slashes** - On Windows, you can use `/` or `\\` in paths
- ✅ **Check for typos** - File names are case-sensitive on Mac/Linux

**Example of correct path format:**
```python
# Good (absolute path)
light_files = ['/Users/yourname/images/eclipse1.fits']

# Also good (relative to notebook location)
light_files = ['./data/eclipse1.fits']

# Bad (will fail)
light_files = ['eclipse1.fits']  # Missing path
```

---

### 2. Plate Solving Fails

**Symptoms:**
- Error: "Plate solving failed" or "No plate solution found"
- Step 1 completes but no plate solution in output

**Solutions:**
- ✅ **Check image quality** - Images need enough stars (at least 10-20 bright stars)
- ✅ **Verify images are not too dark or too bright** - Stars should be visible but not saturated
- ✅ **Check field of view** - Very wide or very narrow fields may be harder to solve
- ✅ **Ensure images overlap** - All images should show the same region of sky
- ✅ **Try different stacking options** - Adjust `sensitive_mode_stack` or `centroid_gaussian_subtract`
- ✅ **Check for Sun/Moon blocking stars** - If Sun/Moon is too large, may not have enough stars

**What to check:**
- Open your stacked image and verify you can see stars
- Count how many stars are visible (need at least 10-20)
- Check that images are properly aligned

---

### 3. Distortion Fitting Fails

**Symptoms:**
- Error: "Distortion fit failed" or "Not enough matched stars"
- Step 2 fails to create output file

**Solutions:**
- ✅ **Verify Step 1 completed successfully** - Check that `centroid_data*.zip` file exists
- ✅ **Check input ZIP file is valid** - Try opening it manually to verify it's not corrupted
- ✅ **Verify observation date is correct** - Wrong date will cause star positions to be off
- ✅ **Check internet connection** - Gaia queries require internet access
- ✅ **Ensure enough stars matched** - Need at least 10-20 stars matched to Gaia
- ✅ **Check star magnitude limits** - Adjust `max_star_mag_dist` if needed (try 11 or 12)

**What to check:**
- Open Step 1 output ZIP and verify it contains star data
- Check that observation date matches when images were taken
- Verify internet connection is working

---

### 4. Eclipse Analysis Gives Unexpected Results

**Symptoms:**
- Deflection coefficient L is very different from 1.751 arcseconds
- Large uncertainty in the measurement
- Error messages about Sun position

**Solutions:**
- ✅ **Verify Step 2 completed successfully** - Check that `distortion_data*.zip` file exists
- ✅ **Check observation date and time are correct** - Critical for calculating Sun position
- ✅ **Verify observation location (lat/long)** - Needed for accurate Sun position calculation
- ✅ **Check that stars are near the Sun** - Need stars within ~10 solar radii for good measurement
- ✅ **Review earlier steps** - Systematic errors in Steps 1-2 will affect Step 3 results
- ✅ **Check limiting magnitude** - May need to adjust `eclipse_limiting_mag` to include more/fewer stars

**What to check:**
- Verify date format: `YYYY-MM-DD` (e.g., `2024-04-08`)
- Verify time format: `HH:MM:SS` in UTC (e.g., `18:30:00`)
- Check latitude/longitude are in degrees (e.g., `40.7128` for New York)
- Review diagnostic plots in output directory

---

### 5. No Stars Matched to Gaia Catalog

**Symptoms:**
- Error: "No stars matched" or "Gaia query returned no results"
- Step 2 fails during Gaia matching phase

**Solutions:**
- ✅ **Check internet connection** - Gaia queries require active internet
- ✅ **Verify observation date is correct** - Wrong date means wrong star positions
- ✅ **Check field has bright enough stars** - Gaia catalog has magnitude limits
- ✅ **Adjust magnitude limits** - Try increasing `max_star_mag_dist` to 12 or 13
- ✅ **Verify plate solution from Step 1** - If plate solve was wrong, Gaia matching will fail
- ✅ **Check for firewall/proxy issues** - Some networks block astroquery connections

**What to check:**
- Test internet connection (try opening a webpage)
- Verify observation date matches image capture date
- Check that Step 1 plate solution looks reasonable

---

### 6. Database Initialization Takes Too Long

**Symptoms:**
- Database initialization hangs or takes >10 minutes
- "Initializing star catalog database..." message doesn't complete

**Solutions:**
- ✅ **Wait patiently** - First-time initialization can take 5-10 minutes (one-time setup)
- ✅ **Check disk space** - Database needs several hundred MB of free space
- ✅ **Restart kernel** - If truly stuck, restart Jupyter kernel and try again
- ✅ **Check for errors** - Look for error messages in the output

**Note:** Database initialization only happens once. After the first run, it should be much faster.

---

### 7. Import Errors

**Symptoms:**
- `ModuleNotFoundError: No module named 'mee2024'`
- Import errors for other packages

**Solutions:**
- ✅ **Activate virtual environment** - Make sure you're in the correct virtual environment
- ✅ **Install MEE2024 package** - Run `pip install -e .` from the MEE2024 directory
- ✅ **Install dependencies** - Run `pip install -r requirements.txt`
- ✅ **Check Python version** - Need Python 3.9 or higher
- ✅ **Restart kernel** - After installing packages, restart Jupyter kernel

**Installation commands:**
```bash
# Activate virtual environment
source mee2024_env/bin/activate  # On Mac/Linux
# or
mee2024_env\Scripts\activate  # On Windows

# Install MEE2024
cd /path/to/MEE2024_HussainKinder
pip install -e .

# Install dependencies
pip install -r requirements.txt
```

---

### 8. Memory Errors

**Symptoms:**
- `MemoryError` or "Out of memory" messages
- Notebook crashes when processing large images

**Solutions:**
- ✅ **Reduce number of images** - Process fewer images at once
- ✅ **Close other programs** - Free up system memory
- ✅ **Use smaller images** - If possible, use lower resolution images
- ✅ **Process in batches** - Run Step 1 multiple times with subsets of images

---

## Getting Help

If you're still having issues:

1. **Check error messages carefully** - They often contain clues about what went wrong
2. **Review the output files** - Check what was created (or not created) at each step
3. **Verify your data** - Make sure your images are valid FITS files with stars visible
4. **Check the original GUI** - Try running the same data through the GUI to compare
5. **Review documentation** - Check the original MEE2024 README and papers

---

## Testing Your Setup

Before running your real data, you can test that everything is working:

1. ✅ **Test imports** - Run the import cell and verify no errors
2. ✅ **Test database** - Database initialization should complete without errors
3. ✅ **Test file validation** - Try validating a small set of test images
4. ✅ **Check output directories** - Verify you can write to the output directory

---

---

# ✅ Testing Checklist

Use this checklist to verify that your notebook is working correctly.

---

## Pre-Run Checks

Before running your analysis:

- [ ] **Virtual environment activated** - You're using the correct Python environment
- [ ] **MEE2024 package installed** - `pip install -e .` completed successfully
- [ ] **Dependencies installed** - All packages from `requirements.txt` are installed
- [ ] **File paths set** - All image file paths are correct and files exist
- [ ] **Output directory set** - You know where results will be saved
- [ ] **Internet connection** - Available for Gaia catalog queries (Step 2)

---

## Step 1: Find Centroids

- [ ] **Imports work** - No `ModuleNotFoundError` when running import cell
- [ ] **Database initializes** - Database preparation completes (may take a few minutes first time)
- [ ] **Files validate** - All input files are found and can be opened
- [ ] **Step 1 completes** - Processing finishes without errors
- [ ] **Output ZIP created** - File `centroid_data{timestamp}.zip` is created
- [ ] **Output file is valid** - ZIP file can be opened and contains expected files
- [ ] **Stars found** - Output shows reasonable number of stars detected (50-200 typical)
- [ ] **Plate solution exists** - Output includes RA, Dec, roll, and plate scale values

**Expected outputs:**
- `centroid_data{timestamp}.zip` (main output)
- `STACKED_CENTROIDS_DATA.csv` (inside ZIP)
- `results.txt` (inside ZIP, contains plate solution)

---

## Step 2: Compute Distortion

- [ ] **Input file found** - Step 1 output ZIP file is detected automatically
- [ ] **Step 2 completes** - Processing finishes without errors
- [ ] **Output ZIP created** - File `distortion_data{timestamp}.zip` is created
- [ ] **Gaia matching works** - Stars are successfully matched to Gaia catalog
- [ ] **Distortion fit succeeds** - Polynomial fit completes without errors
- [ ] **Reasonable fit quality** - RMS error is < 1 arcsecond (check output messages)
- [ ] **Matched stars** - At least 10-20 stars are matched to Gaia

**Expected outputs:**
- `distortion_data{timestamp}.zip` (main output)
- `CATALOGUE_MATCHED_ERRORS.csv` (inside ZIP)
- `distortion_results.txt` (inside ZIP, contains distortion coefficients)

---

## Step 3: Eclipse Analysis

- [ ] **Input file found** - Step 2 output ZIP file is detected automatically
- [ ] **Step 3 completes** - Processing finishes without errors
- [ ] **Output file created** - File `ECLIPSE_OUTPUT{timestamp}.txt` is created
- [ ] **Deflection coefficient calculated** - L value is computed
- [ ] **Reasonable result** - L is close to 1.751 arcseconds (within ±0.5 is good)
- [ ] **Uncertainty reported** - Error/uncertainty value is provided
- [ ] **Diagnostic plots created** - Plots are saved in output directory

**Expected outputs:**
- `ECLIPSE_OUTPUT{timestamp}.txt` (final results)
- `ECLIPSE_OUTPUT{timestamp}/` (directory with plots and diagnostics)

---

## Final Validation

- [ ] **All output files in expected locations** - Check output directory
- [ ] **File sizes reasonable** - Output files are not empty or suspiciously small
- [ ] **Results make sense** - Deflection coefficient L ≈ 1.751 ± 0.5 arcseconds
- [ ] **Plots display correctly** - Diagnostic plots are readable
- [ ] **No error messages** - All steps completed without critical errors
- [ ] **Processing time reasonable** - Total time < 1 hour for typical dataset

---

## Quality Indicators

**Good results:**
- ✅ Deflection coefficient L = 1.751 ± 0.1 arcseconds
- ✅ Distortion fit RMS < 1 arcsecond
- ✅ At least 20 stars used in final analysis
- ✅ Small uncertainty (< 0.2 arcseconds)

**Concerning results:**
- ⚠️ L very different from 1.751 (> 0.5 arcseconds off)
- ⚠️ Large uncertainty (> 0.5 arcseconds)
- ⚠️ Very few stars used (< 10 stars)
- ⚠️ Distortion fit RMS > 2 arcseconds

If you see concerning results, review the troubleshooting guide above.

---

---

# 🧪 Validation and Testing

This section contains validation cells that you can run to verify your setup is working correctly, **even before you have data files ready**.

**✅ Run these cells to test your setup:**

1. **Test 1: Import Validation** - Verifies all packages are installed correctly
2. **Test 2: Configuration Validation** - Checks that configuration structure is correct
3. **Test 3: Function Validation** - Tests helper functions work correctly
4. **Test 4: File Path Validation** - Validates file path formats (without requiring files to exist)
5. **Test 5: Database Initialization Check** - Verifies database module is available

**💡 Tip:** Run these validation tests first to ensure everything is set up correctly before processing your data.

---

### Test 1: Import Validation

This cell verifies that all required packages are installed and can be imported.

In [ ]:
# ============================================================================
# VALIDATION TEST 1: Verify All Imports Work
# ============================================================================
# This test checks that all required packages are installed correctly.
# Run this cell to verify your environment is set up properly.
# ============================================================================

print("=" * 60)
print("TEST 1: IMPORT VALIDATION")
print("=" * 60)
print()

# Track test results
test_results = {}
all_passed = True

# Test standard library imports
print("Testing standard library imports...")
try:
    import os
    import sys
    from pathlib import Path
    import json
    import zipfile
    test_results['standard_library'] = True
    print("  ✅ Standard library imports: PASSED")
except Exception as e:
    test_results['standard_library'] = False
    all_passed = False
    print(f"  ❌ Standard library imports: FAILED - {e}")

# Test scientific computing imports
print("\nTesting scientific computing imports...")
try:
    import numpy as np
    import pandas as pd
    test_results['scientific'] = True
    print("  ✅ Scientific computing imports: PASSED")
    print(f"     NumPy version: {np.__version__}")
    print(f"     Pandas version: {pd.__version__}")
except Exception as e:
    test_results['scientific'] = False
    all_passed = False
    print(f"  ❌ Scientific computing imports: FAILED - {e}")

# Test astronomy imports
print("\nTesting astronomy imports...")
try:
    from astropy.io import fits
    from astropy.coordinates import EarthLocation, SkyCoord
    from astropy.time import Time
    import astropy
    test_results['astronomy'] = True
    print("  ✅ Astronomy imports: PASSED")
    print(f"     Astropy version: {astropy.__version__}")
except Exception as e:
    test_results['astronomy'] = False
    all_passed = False
    print(f"  ❌ Astronomy imports: FAILED - {e}")

# Test MEE2024 package imports
print("\nTesting MEE2024 package imports...")
try:
    from mee2024 import stacker_implementation
    from mee2024 import distortion_fitter
    from mee2024 import eclipse_analysis
    from mee2024 import MEE2024util
    from mee2024 import database_cache
    test_results['mee2024'] = True
    print("  ✅ MEE2024 package imports: PASSED")
except Exception as e:
    test_results['mee2024'] = False
    all_passed = False
    print(f"  ❌ MEE2024 package imports: FAILED - {e}")
    print("     Make sure you've run: pip install -e .")

# Test visualization imports
print("\nTesting visualization imports...")
try:
    import matplotlib.pyplot as plt
    test_results['visualization'] = True
    print("  ✅ Visualization imports: PASSED")
    print(f"     Matplotlib version: {plt.matplotlib.__version__}")
except Exception as e:
    test_results['visualization'] = False
    all_passed = False
    print(f"  ❌ Visualization imports: FAILED - {e}")

# Summary
print()
print("=" * 60)
if all_passed:
    print("✅ ALL IMPORT TESTS PASSED!")
    print("   Your environment is set up correctly.")
else:
    print("❌ SOME IMPORT TESTS FAILED")
    print("   Please install missing packages before proceeding.")
    print("   Run: pip install -r requirements.txt")
print("=" * 60)

### Test 2: Configuration Structure Validation

This cell verifies that the configuration options dictionary has the correct structure and required keys.

In [ ]:
# ============================================================================
# VALIDATION TEST 2: Verify Configuration Structure
# ============================================================================
# This test checks that the options dictionary has all required keys
# and that values are in acceptable ranges.
# ============================================================================

print("=" * 60)
print("TEST 2: CONFIGURATION VALIDATION")
print("=" * 60)
print()

# Check if options dictionary exists
if 'options' not in locals() and 'options' not in globals():
    print("⚠️  WARNING: 'options' dictionary not found.")
    print("   This is expected if you haven't run the configuration cell yet.")
    print("   Run the configuration cell (Cell 4) first, then run this test again.")
else:
    # Required keys that should be in options
    required_keys = [
        'flag_display', 'flag_display2', 'flag_display3',
        'distortionOrder', 'eclipse_method', 'eclipse_limiting_mag',
        'max_star_mag_dist', 'observation_date',
        'catalogue', 'output_dir'
    ]
    
    missing_keys = []
    invalid_values = []
    
    # Check for required keys
    for key in required_keys:
        if key not in options:
            missing_keys.append(key)
    
    # Validate specific values
    if 'distortionOrder' in options:
        valid_orders = ['linear', 'cubic', 'quintic', 'septic']
        if options['distortionOrder'] not in valid_orders:
            invalid_values.append(f"distortionOrder should be one of {valid_orders}, got: {options['distortionOrder']}")
    
    if 'eclipse_method' in options:
        valid_methods = ['Method 1', 'Method 2', 'Method 1 & 2']
        if options['eclipse_method'] not in valid_methods:
            invalid_values.append(f"eclipse_method should be one of {valid_methods}, got: {options['eclipse_method']}")
    
    if 'catalogue' in options:
        valid_catalogues = ['gaia', 'tycho']
        if options['catalogue'] not in valid_catalogues:
            invalid_values.append(f"catalogue should be one of {valid_catalogues}, got: {options['catalogue']}")
    
    # Check numeric ranges
    if 'eclipse_limiting_mag' in options:
        if not isinstance(options['eclipse_limiting_mag'], (int, float)) or options['eclipse_limiting_mag'] < 0 or options['eclipse_limiting_mag'] > 20:
            invalid_values.append(f"eclipse_limiting_mag should be between 0 and 20, got: {options['eclipse_limiting_mag']}")
    
    # Report results
    if missing_keys:
        print("❌ Missing required keys:")
        for key in missing_keys:
            print(f"   - {key}")
    else:
        print("✅ All required configuration keys present")
    
    if invalid_values:
        print("\n⚠️  Invalid configuration values:")
        for val in invalid_values:
            print(f"   - {val}")
    else:
        print("✅ All configuration values are valid")
    
    # Summary
    print()
    print(f"Total configuration keys: {len(options)}")
    if not missing_keys and not invalid_values:
        print("✅ CONFIGURATION VALIDATION PASSED!")
    else:
        print("⚠️  CONFIGURATION VALIDATION: Issues found (see above)")
    
print("=" * 60)

### Test 3: Function Validation

This cell tests that helper functions (like file validation) work correctly.

In [ ]:
# ============================================================================
# VALIDATION TEST 3: Verify Helper Functions Work
# ============================================================================
# This test checks that helper functions are defined and work correctly.
# ============================================================================

print("=" * 60)
print("TEST 3: FUNCTION VALIDATION")
print("=" * 60)
print()

# Test validate_files function
print("Testing validate_files() function...")
if 'validate_files' in locals() or 'validate_files' in globals():
    try:
        # Test with empty list (should return empty list)
        result = validate_files([], 'light')
        if result == []:
            print("  ✅ validate_files() with empty list: PASSED")
        else:
            print(f"  ❌ validate_files() with empty list: FAILED - expected [], got {result}")
        
        # Test with non-existent file (should handle gracefully)
        result = validate_files(['/nonexistent/file.fits'], 'light')
        if isinstance(result, list):
            print("  ✅ validate_files() with non-existent file: PASSED (handles gracefully)")
        else:
            print(f"  ❌ validate_files() with non-existent file: FAILED - expected list, got {type(result)}")
        
        # Test with valid path format (even if file doesn't exist)
        import tempfile
        with tempfile.NamedTemporaryFile(delete=False, suffix='.fits') as tmp:
            tmp_path = tmp.name
            result = validate_files([tmp_path], 'light')
            if isinstance(result, list) and len(result) == 1:
                print("  ✅ validate_files() with valid file: PASSED")
            else:
                print(f"  ⚠️  validate_files() with valid file: Unexpected result")
            import os
            os.unlink(tmp_path)  # Clean up
        
        print("  ✅ validate_files() function: WORKING")
        
    except Exception as e:
        print(f"  ❌ validate_files() function: FAILED - {e}")
else:
    print("  ⚠️  validate_files() function not found")
    print("     This is expected if you haven't run the file validation cell yet.")
    print("     Run the file validation cell (Cell 6) first, then run this test again.")

# Test Path operations
print("\nTesting Path operations...")
try:
    from pathlib import Path
    test_path = Path('/test/path')
    if isinstance(test_path, Path):
        print("  ✅ Path operations: PASSED")
    else:
        print("  ❌ Path operations: FAILED")
except Exception as e:
    print(f"  ❌ Path operations: FAILED - {e}")

# Test ZIP file operations
print("\nTesting ZIP file operations...")
try:
    import zipfile
    import tempfile
    import os
    
    # Create a temporary ZIP file to test operations
    with tempfile.NamedTemporaryFile(delete=False, suffix='.zip') as tmp:
        tmp_zip = tmp.name
    
    with zipfile.ZipFile(tmp_zip, 'w') as z:
        z.writestr('test.txt', 'test content')
    
    # Test reading ZIP
    with zipfile.ZipFile(tmp_zip, 'r') as z:
        files = z.namelist()
        if 'test.txt' in files:
            print("  ✅ ZIP file operations: PASSED")
        else:
            print("  ❌ ZIP file operations: FAILED - cannot read ZIP")
    
    os.unlink(tmp_zip)  # Clean up
    
except Exception as e:
    print(f"  ❌ ZIP file operations: FAILED - {e}")

print()
print("=" * 60)
print("✅ FUNCTION VALIDATION COMPLETE")
print("=" * 60)

### Test 4: File Path Format Validation

This cell validates file path formats without requiring files to actually exist.

In [ ]:
# ============================================================================
# VALIDATION TEST 4: Verify File Path Formats
# ============================================================================
# This test checks that file paths are in the correct format.
# It doesn't require files to exist, just validates the format.
# ============================================================================

print("=" * 60)
print("TEST 4: FILE PATH FORMAT VALIDATION")
print("=" * 60)
print()

# Check if file path variables exist
if 'light_files' in locals() or 'light_files' in globals():
    print("Testing light_files format...")
    if isinstance(light_files, list):
        print("  ✅ light_files is a list: PASSED")
        
        # Check each path format
        invalid_paths = []
        for i, path in enumerate(light_files):
            if path:  # Only check non-empty paths
                # Check if it looks like a path
                if not (isinstance(path, str) and len(path) > 0):
                    invalid_paths.append(f"  Item {i}: Not a valid string")
                elif path.startswith('/') or '\\' in path or '/' in path or path.startswith('./'):
                    # Looks like a path
                    pass
                else:
                    invalid_paths.append(f"  Item {i}: '{path}' doesn't look like a file path")
        
        if invalid_paths:
            print("  ⚠️  Some paths may have issues:")
            for issue in invalid_paths:
                print(f"     {issue}")
        else:
            print("  ✅ All light_files paths are in correct format")
        
        print(f"  Found {len(light_files)} light file(s)")
    else:
        print(f"  ⚠️  light_files is not a list (got {type(light_files)})")
        print("     It should be a list like: ['/path/to/file1.fits', '/path/to/file2.fits']")
else:
    print("⚠️  light_files variable not found")
    print("   This is expected if you haven't run the configuration cell yet.")

# Check dark_files
if 'dark_files' in locals() or 'dark_files' in globals():
    if isinstance(dark_files, list):
        print(f"\n✅ dark_files is a list with {len(dark_files)} file(s)")

# Check flat_files
if 'flat_files' in locals() or 'flat_files' in globals():
    if isinstance(flat_files, list):
        print(f"✅ flat_files is a list with {len(flat_files)} file(s)")

# Check output_directory
if 'output_directory' in locals() or 'output_directory' in globals():
    print(f"\n✅ output_directory is set: '{output_directory if output_directory else '(same as input)'}'")
else:
    print("\n⚠️  output_directory not found")

# Test path operations
print("\nTesting path operations...")
try:
    from pathlib import Path
    import os
    
    # Test absolute path
    test_abs = Path('/test/absolute/path.fits')
    if test_abs.is_absolute():
        print("  ✅ Absolute path detection: PASSED")
    
    # Test relative path
    test_rel = Path('./relative/path.fits')
    if not test_rel.is_absolute():
        print("  ✅ Relative path detection: PASSED")
    
    # Test file extension check
    test_fits = Path('test.fits')
    if test_fits.suffix == '.fits':
        print("  ✅ File extension check: PASSED")
    
except Exception as e:
    print(f"  ❌ Path operations: FAILED - {e}")

print()
print("=" * 60)
print("✅ FILE PATH VALIDATION COMPLETE")
print("=" * 60)
print()
print("💡 Tip: Even if files don't exist yet, you can test the path format.")
print("   When you're ready, add your actual file paths to the configuration cell.")

### Test 5: Database Initialization Check

This cell checks if the database can be initialized (without actually initializing it, to save time).

In [ ]:
# ============================================================================
# VALIDATION TEST 5: Database Initialization Check
# ============================================================================
# This test verifies that the database module is available and can be accessed.
# It does NOT actually initialize the database (to save time).
# ============================================================================

print("=" * 60)
print("TEST 5: DATABASE INITIALIZATION CHECK")
print("=" * 60)
print()

try:
    from mee2024 import database_cache
    
    # Check if database_cache module has the required function
    if hasattr(database_cache, 'prepare_triangles'):
        print("✅ database_cache.prepare_triangles() function is available")
        print("   The database can be initialized when you run the setup cell.")
    else:
        print("❌ database_cache.prepare_triangles() function not found")
    
    # Check if module is properly imported
    print(f"✅ database_cache module imported successfully")
    print(f"   Module location: {database_cache.__file__ if hasattr(database_cache, '__file__') else 'unknown'}")
    
    print()
    print("💡 Note: Database initialization happens in the setup cell (Cell 2).")
    print("   It may take a few minutes the first time, but is much faster afterwards.")
    
except Exception as e:
    print(f"❌ Database check failed: {e}")
    print("   Make sure MEE2024 package is installed: pip install -e .")

print()
print("=" * 60)
print("✅ DATABASE CHECK COMPLETE")
print("=" * 60)

---

# 🎉 Completion Summary

Congratulations! You've reached the end of the MEE2024 analysis notebook.

## What You've Accomplished

If you've successfully completed all three steps, you have:

✅ **Processed your eclipse images** - Stacked, aligned, and found star positions  
✅ **Fitted optical distortion model** - Corrected for telescope/lens aberrations  
✅ **Calculated Einstein's deflection coefficient** - Measured gravitational light deflection  

## Next Steps

### For Your Current Analysis:
1. **Review your results** - Check the final deflection coefficient L value
2. **Compare to Einstein's prediction** - L should be ~1.751 arcseconds
3. **Examine diagnostic plots** - Look in the output directories for visualizations
4. **Document your findings** - Save your results and any notes

### For Future Analyses:
- **Try different datasets** - Process other eclipse images
- **Experiment with settings** - Adjust options to see how they affect results
- **Compare results** - See how different observations compare

### For Coronagraph Adaptation (Future Work):
- This notebook is currently configured for eclipse images
- Future versions will adapt the analysis for coronagraph observations
- The core pipeline structure will remain the same

---

## 📚 Additional Resources

### Documentation:
- **Original MEE2024 README:** See the repository for detailed documentation
- **Scientific Papers:** Check references in the original MEE2024 repository
- **This Notebook:** All explanations and troubleshooting are included above

### Getting Help:
- **Troubleshooting Guide:** See the troubleshooting section above
- **Testing Checklist:** Use the validation tests to diagnose issues
- **Example Workflow:** Review the example workflow section for guidance

### Contributing:
- **Report Issues:** If you find bugs or have suggestions, please report them
- **Improve Documentation:** Help make this notebook even better for others
- **Share Results:** Contribute your findings to the scientific community

---

## 🙏 Acknowledgments

**Original Software:**  
- MEE2024 by Andrew Smith (v0.6.0, May 2024)
- Repository: https://github.com/andrew551/MEE2024

**Notebook Conversion:**  
- Converted from GUI to Jupyter notebook format
- Enhanced with educational comments and explanations
- Designed for learning and educational use

**Scientific Background:**  
- Modern Eddington Experiment (MEE2024)
- Tests Einstein's general theory of relativity
- Measures gravitational light deflection during solar eclipses

---

**Thank you for using the MEE2024 Notebook Version!** 🎉

---

---

## 📋 Example Workflow

This section shows an example of how to use this notebook with sample file paths and expected outputs.

### Example File Structure

Here's an example of how your data might be organized:

```
/path/to/your/data/
├── eclipse_images/
│   ├── eclipse_001.fits
│   ├── eclipse_002.fits
│   ├── eclipse_003.fits
│   └── ...
├── dark_frames/
│   ├── dark_001.fits
│   └── dark_002.fits
├── flat_frames/
│   ├── flat_001.fits
│   └── flat_002.fits
└── output/
    └── (results will be saved here)
```

### Example Configuration

Here's what your configuration cell might look like with real data:

```python
# Example file paths (replace with your actual paths)
light_files = [
    '/path/to/your/data/eclipse_images/eclipse_001.fits',
    '/path/to/your/data/eclipse_images/eclipse_002.fits',
    '/path/to/your/data/eclipse_images/eclipse_003.fits',
]

dark_files = [
    '/path/to/your/data/dark_frames/dark_001.fits',
    '/path/to/your/data/dark_frames/dark_002.fits',
]

flat_files = [
    '/path/to/your/data/flat_frames/flat_001.fits',
    '/path/to/your/data/flat_frames/flat_002.fits',
]

# Output directory
output_directory = '/path/to/your/data/output'
```

### Expected Outputs at Each Step

#### After Step 1:
- **File:** `centroid_data20240101120000.zip`
- **Location:** Your output directory (or same as input if not specified)
- **Contains:**
  - `STACKED_CENTROIDS_DATA.csv` - Star positions
  - `results.txt` - Plate solution (JSON format)
  - Stacked FITS image

#### After Step 2:
- **File:** `distortion_data20240101120000.zip`
- **Location:** Same directory as Step 1 output
- **Contains:**
  - `CATALOGUE_MATCHED_ERRORS.csv` - Matched stars with errors
  - `distortion_results.txt` - Distortion coefficients (JSON format)

#### After Step 3:
- **File:** `ECLIPSE_OUTPUT20240101120000.txt`
- **Location:** Same directory as Step 2 output
- **Contains:**
  - Deflection coefficient L value
  - Uncertainty/error
  - Number of stars used
  - Diagnostic information

### How to Interpret Results

#### Good Results:
- ✅ **Deflection coefficient L ≈ 1.751 ± 0.1 arcseconds**
- ✅ **Distortion fit RMS < 1 arcsecond**
- ✅ **At least 20 stars used in final analysis**
- ✅ **Small uncertainty (< 0.2 arcseconds)**

#### Concerning Results:
- ⚠️ **L very different from 1.751** (> 0.5 arcseconds off)
  - Check observation date/time
  - Verify earlier steps completed correctly
  - Review diagnostic plots
  
- ⚠️ **Large uncertainty** (> 0.5 arcseconds)
  - May need more stars
  - Check data quality
  - Verify distortion fit quality

- ⚠️ **Very few stars** (< 10 stars)
  - Check limiting magnitude settings
  - Verify stars are near the Sun
  - Review Step 1 and Step 2 outputs

### Processing Time Estimates

For a typical eclipse dataset:

- **Step 1:** 5-15 minutes (depends on number of images)
- **Step 2:** 10-30 minutes (depends on number of stars and internet speed)
- **Step 3:** 5-15 minutes
- **Total:** ~30-60 minutes for complete analysis

### Tips for First-Time Users

1. **Start with a small test dataset** - Use 5-10 images first to verify everything works
2. **Check file paths carefully** - Use absolute paths to avoid confusion
3. **Monitor output messages** - They provide useful information about progress
4. **Save your work** - Notebook state and output files should be saved regularly
5. **Review diagnostic plots** - They help identify issues early

---